# Download and Extract data and model 

In [13]:
# Download data
!gdown https://drive.google.com/uc?id=1bmCaH8qvdgDDTv9AUTdCnRqoj6tKYcqG

# Download MaskRCNN saved model
!gdown https://drive.google.com/uc?id=1hOfoTwa_dE1I8PcDCRkcTIn9cXOVPx7U

^C


In [2]:
import patoolib

# Extract data
patoolib.extract_archive("./InstanceSeg_ValSplit_data.rar", outdir="./")

# Extract model
patoolib.extract_archive("./InstanceSeg_ValSplit_data.rar", outdir="./")

patool: Extracting ./InstanceSeg_ValSplit_data.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- c:\Users\Abdelrahman\Documents\GitHub\Intance-Seg-Eval\InstanceSeg_ValSplit_data.rar
patool:     with cwd=./
patool: ... ./InstanceSeg_ValSplit_data.rar extracted to `./'.


'./'

# Imports

In [3]:
import json
import numpy as np
from tqdm import tqdm
import tensorflow as tf

In [4]:
from utils.segment.metrics import Metrics

In [5]:
from utils.utils import *

# Load data and instantiate COCO class

In [6]:
# read data
with open('data/random_coco_split_val.json','r') as f:
    data= json.load(f)

In [7]:
# COCO class
coco= COCO(data)

# create lists of image_IDs, image_filenames
image_IDs, image_filenames= coco.get_images_IDS_filenames()  

# create annotaions list
annotations= coco.get_annotaions()

# Load model

In [8]:
# define model
model= tf.saved_model.load('./saved_model')

# Loop over images and calculate stats used for metric calculations

In [9]:
stats=[]
for i in tqdm(range(len(image_IDs))):
    ####################################################################### GT #######################################################################
    # Get crrent image ID and filename
    current_image_ID= image_IDs[i]
    current_Image_name= image_filenames[i]

    # from annotations list: get current image annotation
    current_image_annotations= coco.retrieve_image_GT(current_image_ID) 

    # Get GT_masks and labels
        # GT_masks: array[M, H, W], where M is the number of masks.
        # labels: array[M, 5], normalized(class, y1, x1, y2, x2)
    GT_masks, labels=single_image_anno_Processing(current_image_annotations, (1920, 1080))              

    ####################################################################### PRED #######################################################################
    # Get PRED for a current image
    image_arr= image_file_to_tensor(f'./data/445249_Road_distress/ds0/images/{current_Image_name}')            
    batched_tensor =image_arr_to_batched_tensor(image_arr)                                                  
    detections= model(batched_tensor)
    detec, PRED_masks, boxes, classes, scores= get_detection_and_pred_masks(detections, image_arr)            

    ####################################################################### PostProcess #######################################################################
    GT_masks, PRED_masks= resize_masks(GT_masks, PRED_masks, (512, 960))     # resizes input masks to shape
    reshaped_GT_masks, reshaped_PRED_masks= reshape_masks(GT_masks, PRED_masks)   # reshape masks to shape (N,n) where: N=number of mask instances, n= imageH x imageW

    ####################################################################### Compute metrics #######################################################################

    # define iouv vector
    iouv = np.linspace(0.5, 0.95, 10) 
    niou = iouv.size

    nl, npr = labels.shape[0], detec.shape[0]  # number of labels, predictions
    correct_masks = np.zeros((npr, niou), dtype=bool)  # init
    correct_bboxes = np.zeros((npr, niou), dtype=bool)  # init

    if npr == 0:
        if nl:
            stats.append((correct_masks, correct_bboxes,*np.zeros((2, 0)), labels[:, 0]))

    if nl:
        # Get correct masks and boxes
        correct_masks= process_batch(detec, labels, iouv, pred_masks=reshaped_PRED_masks, gt_masks=reshaped_GT_masks, masks= True)
        correct_bboxes= process_batch(detec, labels, iouv, masks= False)
    stats.append((correct_masks, correct_bboxes, detec[:, 4], detec[:, 5], labels[:, 0]))  # (conf, pcls, tcls)
    

100%|██████████| 327/327 [03:56<00:00,  1.38it/s]


# Get class names 

In [10]:
names= get_names_dict(coco.get_categories())                 
names

{1: 'Longitudinal crack',
 2: 'Transverse crack',
 3: 'Block crack',
 4: 'Bleeding-',
 5: 'Aligator crack',
 6: 'Pothole'}

# Compute metrics

In [11]:
def compute_Mrtrics(stats, names, no_of_eval_images):
    # process stats
    stats = [np.concatenate(x) for x in zip(*stats)]
    # instantiate a metrics class
    metrics = Metrics()
    
    # Compute metrics
    if len(stats) and stats[0].any():
        results = ap_per_class_box_and_mask(*stats, plot=False, save_dir='.', names=names)
        metrics.update(results)

    pf = '%22s' + '%11i' * 2 + '%11.3g' * 8  # print format
    nt = np.bincount(stats[4].astype(int), minlength=4)  # number of targets per class

    # print mean results
    print(('%22s' + '%11s' * 2 + '%11s' * 8) % ("Class", "Images", "Instances", "Box(P", "R", "mAP50", "mAP50-95)", "Mask(P", "R", "mAP50", "mAP50-95)"))
    
    # print per class results
    print(pf % ("all", no_of_eval_images, nt.sum(), *metrics.mean_results()))
    for i, c in enumerate(metrics.ap_class_index):
        print(pf % (names[c], no_of_eval_images, nt[c], *metrics.class_result(i)))

In [12]:
compute_Mrtrics(stats= stats, names=names, no_of_eval_images= len(image_IDs))                         

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95)
                   all        327       1245      0.829      0.765      0.835      0.588      0.815      0.709      0.791       0.45
    Longitudinal crack        327        786      0.978      0.911      0.954      0.866      0.947      0.879      0.922      0.401
      Transverse crack        327        312      0.925      0.808      0.914      0.642      0.928      0.784      0.884      0.396
           Block crack        327         55      0.937      0.727      0.856      0.718      0.952      0.719      0.856      0.758
             Bleeding-        327         58      0.843      0.569      0.719      0.381      0.816        0.5      0.662       0.35
        Aligator crack        327         23      0.567       0.87      0.849      0.547      0.665      0.862      0.849      0.542
               Pothole        327         11       0.72      0.705   